In [26]:
import numpy as np
import pandas as pd
import os
import SimpleITK as sitk
from radiomics import featureextractor
import gc
import glob
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler

In [4]:
def find_t2W_files(dir, fileinfo):
    dir = os.path.normpath(dir)
    pattern = f"**/*{fileinfo}"  # Recursive glob pattern
    search_path = os.path.join(dir, pattern)
    t2w_file_paths = glob.glob(search_path, recursive=True)
    return t2w_file_paths

def get_path_by_patient_id(df, patient_id, column_name):
    """
    Function to get the path based on patient_id.
    
    Args:
    df (pd.DataFrame): The DataFrame containing the data.
    patient_id (str): The patient ID to search for.
    column_name (str): The name of the column to retrieve the path from.
    
    Returns:
    str: The path if found, otherwise None.
    """
    if column_name not in df.columns:
        raise KeyError(f"Column '{column_name}' does not exist in the DataFrame.")
    
    # Convert patient_id to string to match the DataFrame's patient_id type
    patient_id = str(patient_id)
    
    # Check if the patient_id exists in the DataFrame
    if patient_id not in df['patient_id'].values:
        print(f"patient_id {patient_id} does not exist in the DataFrame.")
        return None
    
    result = df.loc[df['patient_id'] == patient_id, column_name]
    if not result.empty:
        return result.values[0]
    else:
        return None

In [21]:
fileInfoT2w = '_t2w_resampled.mha'

fileInfo = '.nii.gz'
t2w_dir = r"Data\t2w_spacing_resampled2"
guerbet_dir = r"Data\Guerbet23_resampled2"
t2w_paths = find_t2W_files(t2w_dir, fileInfoT2w)
Guerbet23_nii_gz_paths = find_t2W_files(guerbet_dir, fileInfo)

t2w_patient_ids = [path.split('\\')[-1].split("_")[0] for path in t2w_paths]
Guerbet23_patient_ids = [path.split('\\')[-1].split("_")[0] for path in Guerbet23_nii_gz_paths]
t2w_df = pd.DataFrame({'patient_id': t2w_patient_ids, 'T2w_path': t2w_paths})
Guerbet23_df = pd.DataFrame({'Guerbet23_path': Guerbet23_nii_gz_paths, 'patient_id': Guerbet23_patient_ids})
df_path_merged_Guerbet23 = pd.merge(t2w_df, Guerbet23_df, on='patient_id')

def combine_images(Guerbert23_path, Human_path):

    photo = sitk.ReadImage(Guerbert23_path)
    overlay = sitk.ReadImage(Human_path)

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(photo)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    overlay = resampler.Execute(overlay)
    overlay_resampled = resampler.Execute(overlay)
    combined_image = sitk.Cast(photo, sitk.sitkFloat32) + sitk.Cast(overlay_resampled, sitk.sitkFloat32)
    return combined_image

def check_mask(mask):
    mask_array = sitk.GetArrayFromImage(mask)
    unique_values = np.unique(mask_array)
    print(f"Unique values in mask: {unique_values}")
    if len(unique_values) <= 1:
        raise ValueError("No labels found in this mask (i.e. nothing is segmented)!")
    
def resample_mask_to_image(mask, image):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(image)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    resampler.SetDefaultPixelValue(0)
    resampled_mask = resampler.Execute(mask)
    return resampled_mask


def align_image_and_mask(image, mask):
    mask.SetOrigin(image.GetOrigin())  # Force same origin
    mask.SetSpacing(image.GetSpacing())  # Ensure same spacing
    mask.SetDirection(image.GetDirection())  # Align orientation
    return mask
    
def extract_intensity_features(mha_folder0):
    """Extracts first-order intensity, 2D shape, and 3D shape intensity features from all .mha files in a given folder using PyRadiomics."""

    if not os.path.exists(mha_folder0):
        print(f"ERROR: Folder {mha_folder0} does not exist!")
        return
    settings_path = os.path.join('Params.yaml')
    
    extractor = featureextractor.RadiomicsFeatureExtractor(settings_path)

    data = []

    for patient_img in os.listdir(mha_folder0):
        if patient_img.endswith(".mha"):
            image_path = os.path.join(mha_folder0, patient_img)
            patient_id = patient_img.split("_")[0]
        
        else:
            print(f"Skipping {patient_id}: No .mha file found.")
            continue
        
        try: 
            Guerbet_path = get_path_by_patient_id(Guerbet23_df, patient_id, 'Guerbet23_path')
            if Guerbet_path is None:
                print(f"⚠️ Skipping {image_path}: Mask file {Guerbet_path} not found!")
                continue 
            

            image = sitk.ReadImage(image_path)
            mask = sitk.ReadImage(Guerbet_path)
            #mask.CopyInformation(image)
            
            features = extractor.execute(image, mask)

            feature_row = {"patient_id": patient_id}
            mesh_volume = features.get("original_shape_MeshVolume", None)
            max_diameter = features.get("original_shape_Maximum3DDiameter", None)
            max2D_diameter = features.get("original_shape_Maximum2DDiameterSlice", None)

            if mesh_volume is not None and max_diameter is not None and max_diameter != 0:
                feature_row["Diameter-Volum-ratio"] = max_diameter / mesh_volume
            else:
                feature_row["Diameter-Volum-ratio"] = None

            if mesh_volume is not None and max2D_diameter is not None and max2D_diameter != 0:
                feature_row["Diameter-Volum-ratio-2D"] = max2D_diameter / mesh_volume
            else:
                feature_row["Diameter-Volum-ratio-2D"] = None

            for key, value in features.items():
                if key.startswith("original"):  # Filter out metadata keys
                    feature_row[key] = value
                    #print(key, value)
            data.append(feature_row)
        
        except Exception as e:
            print(f"Error processing {patient_id}: {e}")
    return data

In [ ]:
data = []
mha_folder0 = r"Data/t2w_spacing_resampled2/resampled0"
mha_folder1 = r"Data/t2w_spacing_resampled2/resampled1"
mha_folder2 = r"Data/t2w_spacing_resampled2/resampled2"
mha_folder3 = r"Data/t2w_spacing_resampled2/resampled3"
mha_folder4 = r"Data/t2w_spacing_resampled2/resampled4"
data1 = extract_intensity_features(mha_folder0)
data.extend(data1)
data2 = extract_intensity_features(mha_folder1)
data.extend(data2)
data3 = extract_intensity_features(mha_folder2)
data.extend(data3)
data4 = extract_intensity_features(mha_folder3)
data.extend(data4)
data5 = extract_intensity_features(mha_folder4)
data.extend(data5)
df = pd.DataFrame(data)
df.to_csv("Data/Radiomic_features_Radiomic_features.csv")
print("\n🎉 Feature extraction complete! Data saved to 'Radiomic_features.csv'.")

gc.collect()

In [ ]:
df = pd.read_csv("Data/Radiomic_features_Radiomic_features.csv")

# Round all float columns to 3 decimal places (you can change to 2 if needed)
df_rounded = df.round(3)

# Save the new file (overwrite or save as new)
df_rounded.to_csv("Data/Radiomic_features/radiomics_features_rounded.csv", index=False)

In [ ]:
feature_mapping = {
        "Volume": "original_shape_MeshVolume",
        "Surface_area": "original_shape_SurfaceArea",
        "Diameter": "original_shape_Maximum3DDiameter",
        "Histogram_based": "original_firstorder_Uniformity",
        "Diameter_to_volume_ratio": "Diameter-Volum-ratio",
        "Diameter_to_volume_ratio_2D": "Diameter-Volum-ratio-2D",
        "Mean intensity": "original_firstorder_Mean",
        "Skweness": "original_firstorder_Skewness",
        "Kurtosis": "original_firstorder_Kurtosis",
        "Entropy": "original_firstorder_Entropy",
        "Sphericity": "original_shape_Sphericity",
        "Elongation": "original_shape_Elongation",
        "Compactness": "original_shape_Compactness1",
        "Surface_Area_to_Volume_ratio": "original_shape_SurfaceVolumeRatio",
        "GLCM": "original_glcm_Contrast",
        "GLRLM": "original_glrlm_RunEntropy",
        "GLSZM": "original_glszm_ZoneEntropy",
        "NGTDM_Coarseness": "original_ngtdm_Coarseness",
        "NGTDM_Contrast": "original_ngtdm_Contrast"

    }

guidelines = [
        {"guideline": "Signal intensity: T2WI guidelines mention that hypointense lesions are often high-grade and indicative of potential malignancy.",
         "feature_name": "Mean intensity", "feature_context": "lower indicates hypointense"},
         {"guideline": "Signal intensity: T2WI guidelines mention that hypointense lesions are often high-grade and indicative of potential malignancy.",
         "feature_name": "Skweness", "feature_context": "negative indicates dark lesions"},
         {"guideline": "Signal intensity: T2WI guidelines mention that hypointense lesions are often high-grade and indicative of potential malignancy.",
          "feature_name": "Kurtosis", "feature_context": "highly concentrated lesions in a small intensity range"},
         {"guideline": "Signal intensity: T2WI guidelines mention that hypointense lesions are often high-grade and indicative of potential malignancy.",
          "feature_name": "Histogram_based", "feature_context": "Histogram based features can provide information on the distribution of intensity values"},
         {"guideline": "Tumor size: Lesion size is an important factor in determining risk. Lesions larger than 1 cm are more likely to be significant.",
          "feature_name": "Volume", "feature_context": "larger lesions have higher volumes"},
         {"guideline": "Tumor size: Lesion size is an important factor in determining risk. Lesions larger than 1 cm are more likely to be significant.",
          "feature_name": "Surface_area", "feature_context": "correlated to volume"},
         {"guideline": "Tumor size: Lesion size is an important factor in determining risk. Lesions larger than 1 cm are more likely to be significant.",
          "feature_name": "Diameter", "feature_context": "Diameter of the lesion"},
         {"guideline": "Tumor size: Lesion size is an important factor in determining risk. Lesions larger than 1 cm are more likely to be significant.",
          "feature_name": "Diameter_to_volume_ratio", "feature_context": "Ratio between diameter and volume"},
          {"guideline": "Tumor size: Lesion size is an important factor in determining risk. Lesions larger than 1 cm are more likely to be significant.",
           "feature_name": "Diameter_to_volume_ratio_2D", "feature_context": "Ratio between diameter and volume in 2D"},
         {"guideline": "Shape of the tumor: Irregular or lenticular shapes are more likely to be associated with high-grade tumors.",
          "feature_name": "Sphericity", "feature_context": "Low for irregularity"},
         {"guideline": "Shape of the tumor: Irregular or lenticular shapes are more likely to be associated with high-grade tumors.",
          "feature_name": "Elongation", "feature_context": "high can indicate more irregularity"},
         {"guideline": "Shape of the tumor: Irregular or lenticular shapes are more likely to be associated with high-grade tumors.",
          "feature_name": "Compactness", "feature_context": "how close the lesion is to being spherical - non-compact is often associated with more aggressive"},
         {"guideline": "Shape of the tumor: Irregular or lenticular shapes are more likely to be associated with high-grade tumors.",
          "feature_name": "Shape_index", "feature_context": "Shape of the index"},
         {"guideline": "Margins of the tumor: Poorly defined margins indicate a higher likelihood of extraprostatic extension (EPE).",
          "feature_name": "Surface_Area_to_Volume_ratio", "feature_context": "higher are likely to have poor defined boundaries"},
         {"guideline": "Margins of the tumor: Poorly defined margins indicate a higher likelihood of extraprostatic extension (EPE).",
          "feature_name": "Entropy", "feature_context": "high indicates heterogeneity which means poorly defined margins"},
         {"guideline": "Margins of the tumor: Poorly defined margins indicate a higher likelihood of extraprostatic extension (EPE).",
          "feature_name": "GLCM", "feature_context": "e.g contrast indicates heterogeneous intensity"},
         {"guideline": "Extraprostatic extension: Indicates advanced disease with infiltration beyond the prostate.",
          "feature_name": "GLSZM", "feature_context": "(Gray-level size zone matrix), texture variations at the tumor boundary, which can be indicative of extraprostatic extension"},
         {"guideline": "Extraprostatic extension: Indicates advanced disease with infiltration beyond the prostate.",
          "feature_name": "GLRLM", "feature_context": "(Gray-level run length), texture variations at the tumor boundary, which can be indicative of extraprostatic extension"},
         {"guideline": "Extraprostatic extension: Indicates advanced disease with infiltration beyond the prostate.",
          "feature_name": "NGTDM_Coarseness", "feature_context": "Lower values indicate sharper transitions and finer edges. Higher values suggest irregular borders (more coarse and with higher contrast) and also “more disorganized tissue”, which can be correlated to more invasive tumors. "},
          {"guideline": "Extraprostatic extension: Indicates advanced disease with infiltration beyond the prostate.",
          "feature_name": "NGTDM_Contrast", "feature_context": "Lower values indicate sharper transitions and finer edges. Higher values suggest irregular borders (more coarse and with higher contrast) and also “more disorganized tissue”, which can be correlated to more invasive tumors. "}    
    ]

def create_guidelines_csv(
    features_csv_path,
    output_path,
    include_guideline=True,
    include_feature_name=True,
    include_feature_value=True,
    include_feature_context=True
):
    # Load feature data
    features_df = pd.read_csv(features_csv_path)

    rows = []

    for _, patient_row in features_df.iterrows():
        patient_id = int(patient_row["patient_id"])

        for guideline in guidelines:
            feature_name = guideline["feature_name"]
            csv_column = feature_mapping.get(feature_name)

            # Skip if the mapped feature column is missing
            if not csv_column or csv_column not in patient_row:
                continue

            feature_value = patient_row[csv_column]
            row = {
                "patient_id": patient_id
            }

            if include_guideline:
                row["guideline"] = guideline["guideline"]
            if include_feature_name:
                row["feature_name"] = feature_name
            if include_feature_value:
                row["feature_value"] = feature_value
            if include_feature_context:
                row["feature_context"] = guideline.get("feature_context", "")

            rows.append(row)
    guidelines_df = pd.DataFrame(rows)
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    guidelines_df.to_csv(output_path, index=False)
    print(f"\n✅ CSV file created and saved to: '{output_path}' with {len(rows)} rows.")


In [ ]:
features_csv = "Data/Radiomic_features/radiomics_features_rounded.csv"
output_path = "Data/Guidelines/guidelines_features_rounded.csv"
create_guidelines_csv(features_csv, output_path, 
                      include_guideline=True,
                      include_feature_name=True,
                      include_feature_value=True,
                      include_feature_context=True)


✅ CSV file created and saved to: 'guidelines_features_rounded.csv' with 25156 rows.


In [ ]:
file_path = "Data/Radiomic_features/Radiomic_features.csv"  # Update this if needed
df = pd.read_csv(file_path)

# Extract patient ID and drop non-feature columns
patient_ids = df["patient_id"]
df2 = df.drop(columns=["patient_id", "Unnamed: 0"])
X = df2.copy()

# Initialize scalers
scalers = {
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "RobustScaler": RobustScaler(),
    "MaxAbsScaler": MaxAbsScaler()
}

# Apply scalers
scaled_datasets = {}

for name, scaler in scalers.items():
    X_scaled = scaler.fit_transform(X)
    scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

    # Add patient_id back
    scaled_df.insert(0, "patient_id", patient_ids)

    scaled_datasets[name] = scaled_df
    print(f"{name} applied. Shape: {scaled_df.shape}")

# Optionally save scaled versions to CSV
for name, df_scaled in scaled_datasets.items():
    output_file = f"scaled_radiomic_features_{name}.csv"
    df_scaled.to_csv(output_file, index=False)
    
    # Call your guideline generator function
    create_guidelines_csv(
        output_file, 
        f"Data/Guidelines/guidelines_scaled_features_{name}.csv", 
        include_guideline=True,
        include_feature_name=True,
        include_feature_value=True,
        include_feature_context=True
    )

StandardScaler applied. Shape: (1324, 21)
MinMaxScaler applied. Shape: (1324, 21)
RobustScaler applied. Shape: (1324, 21)
MaxAbsScaler applied. Shape: (1324, 21)

✅ CSV file created and saved to: 'guidelines_scaled_features_StandardScaler.csv' with 25156 rows.

✅ CSV file created and saved to: 'guidelines_scaled_features_MinMaxScaler.csv' with 25156 rows.

✅ CSV file created and saved to: 'guidelines_scaled_features_RobustScaler.csv' with 25156 rows.

✅ CSV file created and saved to: 'guidelines_scaled_features_MaxAbsScaler.csv' with 25156 rows.
